<a href="https://colab.research.google.com/github/675-lqr/CLIP/blob/main/%E5%9F%BA%E4%BA%8ECLIP%E7%9A%84AI%E7%94%9F%E5%9B%BE%E5%B1%9E%E6%80%A7%E5%B1%A5%E7%BA%A6_%2B_%E8%AF%AF%E5%AF%BC%E5%81%8F%E5%90%91%E8%81%94%E5%90%88%E8%AF%84%E6%B5%8Bpipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **配置环境与模型下载**

In [3]:
!nvidia-smi
!pip -q install open-clip-torch ftfy regex tqdm pillow pandas

/bin/bash: line 1: nvidia-smi: command not found


In [1]:
!pip -q install open-clip-torch ftfy regex tqdm pillow pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00


In [4]:
! pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-aybl9jyn
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-aybl9jyn
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=d3d9135ab22dfcc661ada7440947c89e3445de54b12753821deec0964c7bf9d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-qxytbq_4/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [5]:
import os, json
import torch, pandas as pd, numpy as np
from PIL import Image
from tqdm.auto import tqdm
import open_clip

device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "ViT-L-14"
PRETRAINED = "laion2b_s32b_b82k"   # 多语更稳；想对齐OpenAI原版可用 "openai"

model, _, preprocess = open_clip.create_model_and_transforms(
    MODEL_NAME, pretrained=PRETRAINED, device=device
)
tokenizer = open_clip.get_tokenizer(MODEL_NAME)
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-23): 24 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1024,), eps=1e-05, elementwi

# **定义“文化类别语义原型”（CLASS_DEFS）**

这里的文字用于让 CLIP“理解类别”，请尽量用结构/部件特征（区分度高），避免“4k, highly detailed”这类风格词。

给每个类准备 2–5 条含义近似但用词不同的句子，以提高鲁棒性：每个类别的最终向量 = 所有定义句 × 模板句的平均；这样 CLIP 在比较图片时不会被某一条语句“绑偏”

In [7]:
# 每个类别写若干“判别性描述句”（不是生成时的原始 prompt，而是这个文化符号的定义特征）
CLASS_DEFS = {
    "cn_dragon": [
        "a traditional Chinese dragon (Long), serpentine body, no wings, whiskers, horned head, five claws, auspicious clouds",
        "an imperial Chinese dragon, snake-like long body, no wings, whiskers, antler-like horns, five claws, cloud motifs",
    ],
    "western_dragon": [
        "a western dragon with large bat-like wings, four legs, bulky body, fire-breathing",
        "a european dragon, big wings, massive body, four limbs, lizard-like",
    ],
    "wyvern": [
        "a wyvern, two-legged winged dragon, bat-like wings, no forelegs",
        "a flying wyvern with two legs and wings instead of arms",
    ],
    "phoenix": [
        "a phoenix with fiery feathers, mythic bird, flame-like plumage",
        "a mythical firebird with radiant feathers and long tail",
    ],
    "hanbok": [
        "traditional Korean hanbok with jeogori jacket and chima skirt, voluminous silhouette, goreum ribbon",
        "Korean hanbok outfit, jeogori top, chima skirt, norigae accessory",
    ],
    "hanfu": [
        "traditional Chinese hanfu, cross-collar (jiaoling youren) robe, wide sleeves, layered garments",
        "hanfu clothing, crossed collar, long robe, flowing wide sleeves",
    ],
    "kimono": [
        "traditional Japanese kimono with obi sash, straight-line silhouette, wide sleeves",
        "Japanese kimono outfit with obi belt tied at the back, decorative patterns",
    ],
}

# **定义“属性探针”做属性履约度**

require_present 表示该类别应当出现的属性；
require_absent 表示不应出现的属性（如中式龙的“无翅”）；
这些句子越清晰、越短、越“可判别”，分数越稳定。

In [8]:
# 针对“目标类别”，我们拆成若干属性（正/反），以量化 prompt 履约度
ATTR_SCHEMA = {
    "cn_dragon": {
        "require_present": {
            "serpentine_body": "a dragon with a long serpentine snake-like body",
            "whiskers": "a dragon with long whiskers",
            "horned_head": "a dragon with antler-like horned head",
            "five_claws": "a dragon with five claws on each foot",
            "auspicious_clouds": "auspicious cloud motifs surrounding the dragon",
        },
        "require_absent": {
            "wings": "a dragon with large bat-like wings",
            "bulky body": "massive body, four limbs, lizard-like"
        },
    },
    "hanbok": {
        "require_present": {
            "jeogori_chima": "hanbok consisting of a jeogori top and a chima skirt",
            "goreum": "a hanbok with a ribbon goreum tie at the front",
            "voluminous_silhouette": "a hanbok with a voluminous silhouette",
        },
        "require_absent": {}
    }
}

# 反例句：用于把“有/无”对立成二分类 margin（可继续完善）
NEG_SENTENCE_MAP = {
    "serpentine_body": "a dragon with a short bulky body",
    "whiskers": "a dragon without whiskers",
    "horned_head": "a dragon without horns on its head",
    "five_claws": "a dragon with three claws only",
    "auspicious_clouds": "no cloud motifs around the dragon",
    "wings": "a dragon without wings",
}

# **编码辅助：模板集成、文本/图像编码函数python复制代码**

模板能提升鲁棒性（多句合成一个类的“原型向量”）。
图像与文本都做了归一化，便于用点积等价余弦相似度。

In [9]:
TEMPLATES = [
    "an illustration of {}",
    "a detailed depiction of {}",
    "a high-quality image of {}",
    "a traditional artwork of {}",
]

def encode_texts(texts):
    with torch.no_grad():
        tok = tokenizer(texts).to(device)
        feat = model.encode_text(tok)
        feat = feat / feat.norm(dim=-1, keepdim=True)
    return feat

def pooled_proto(sentences):
    # 模板扩增 → 编码 → 均值池化 → 归一化
    variants = []
    for s in sentences:
        for tpl in TEMPLATES:
            variants.append(tpl.format(s))
    feat = encode_texts(variants)
    pooled = feat.mean(dim=0, keepdim=True)
    return pooled / pooled.norm(dim=-1, keepdim=True)

def load_image(path):
    im = Image.open(path).convert("RGB")
    return preprocess(im)

def clip_image_feat(img_path):
    with torch.no_grad():
        img = load_image(img_path).unsqueeze(0).to(device)
        im = model.encode_image(img)
        im = im / im.norm(dim=-1, keepdim=True)
    return im

# **构建“类别向量库”（用于闭集分类）**

每个类对应一个“语义原型向量”，后续把图片向量与它们做相似度比较即可分类。

In [10]:
# 如果你想暂时只研究“龙”的问题，可以这样手动指定：
use_classes = ["cn_dragon", "western_dragon"]

# 如果你想同时包含“双足飞龙”也行：
# use_classes = ["cn_dragon", "western_dragon", "wyvern"]

# 检查：确保这些类名在 CLASS_DEFS 中都有定义
for c in use_classes:
    assert c in CLASS_DEFS, f"Class {c} not found in CLASS_DEFS"

In [11]:
# 用这些类构建类别语义向量库
class_names = use_classes
class_bank = torch.cat([pooled_proto(CLASS_DEFS[c]) for c in class_names], dim=0)

print("Classes used in this experiment:", class_names)
print("Text bank shape:", class_bank.shape)

Classes used in this experiment: ['cn_dragon', 'western_dragon']
Text bank shape: torch.Size([2, 768])


# **读取 CSV文件**

**这里需要改路径**
如果csv里提供了 content_label，后面会自动并入分析

csv文件表头：image_path, intended_label, intended_prompt, content_label（可选）

intended_label：目标类别（将参与“类别偏差率”评估）。

intended_prompt：给生成器的英文 prompt 原文（用于记录，不直接喂给 CLIP 分类；属性检测用具体文化特征定义的“判别句”更稳）。

（可选） content_label 做人工标注的“实际是什么”，方便区分“生成器跑题 vs CLIP误判”。

In [13]:
CSV_PATH = "/content/test.csv"   # ← 把它改成你的真实路径
df = pd.read_csv(CSV_PATH)

required_cols = {"image_path","intended_label","intended_prompt"}
assert required_cols.issubset(df.columns), f"CSV must have columns: {required_cols}"
df["image_path"] = df["image_path"].astype(str)

# 简单路径检查（可忽略告警）
missing = [p for p in df["image_path"] if not os.path.exists(p)]
if missing:
    print("⚠️ 以下路径找不到（请检查路径是否为相对路径如 'image/xxx.jpg'）：")
    for m in missing[:10]:
        print(" -", m)
    if len(missing) > 10:
        print(f"... 以及 {len(missing)-10} 个")

# intended_label 必须在 CLASS_DEFS 中
valid_mask = df["intended_label"].isin(class_names)
if not valid_mask.all():
    bad = df[~valid_mask]["intended_label"].unique().tolist()
    raise ValueError(f"以下 intended_label 不在 CLASS_DEFS 中: {bad}")

# **逐图计算：类别预测 + 属性履约度 + 西化指数**

clip_pred 是 CLIP 看来更像哪个类；
_adherence 是“属性履约均分”；
westernization_index > 0 表示更像西式龙，< 0 更像中式龙（只对中式龙目标给出）。

In [14]:
def classify_image(im):
    sims = (im @ class_bank.T)[0]                # 与各类别相似度
    pred_idx = int(torch.argmax(sims).item())
    pred = class_names[pred_idx]
    probs = torch.softmax(sims, dim=-1).cpu().numpy()
    return pred, probs, sims.cpu().numpy()

def attribute_scores(im, intended_label):
    schema = ATTR_SCHEMA.get(intended_label, None)
    if schema is None:
        return {}
    results = {}

    # 汇总“正/反”句
    pos_texts, neg_texts, keys, sign = [], [], [], []
    for k, pos in schema.get("require_present", {}).items():
        neg = NEG_SENTENCE_MAP.get(k, f"not {pos}")
        pos_texts.append(pos); neg_texts.append(neg); keys.append(k); sign.append(+1)
    for k, pos in schema.get("require_absent", {}).items():
        neg = NEG_SENTENCE_MAP.get(k, f"not {pos}")
        pos_texts.append(pos); neg_texts.append(neg); keys.append(k); sign.append(-1)

    if not keys:
        return {}

    pos_feat = encode_texts(pos_texts)
    neg_feat = encode_texts(neg_texts)

    with torch.no_grad():
        sp = (im @ pos_feat.T)[0].cpu().numpy()
        sn = (im @ neg_feat.T)[0].cpu().numpy()

    margins = sp - sn
    probs = 1 / (1 + np.exp(-margins))  # 转 0~1

    for i, k in enumerate(keys):
        val = probs[i] if sign[i] > 0 else (1 - probs[i])  # “应缺失”的属性用反向
        results[k] = float(val)
    if results:
        results["_adherence"] = float(np.mean(list(results.values())))
    return results

rows = []
for _, r in tqdm(df.iterrows(), total=len(df), desc="Scoring"):
    path = r["image_path"]
    intended = r["intended_label"]
    prompt_raw = r["intended_prompt"]

    im = clip_image_feat(path)
    pred, probs, sims = classify_image(im)

    # 仅针对 intended=cn_dragon 给个“西化指数”：sim(western) - sim(cn)
    westernization = None
    if intended == "cn_dragon":
        si = sims[class_names.index("cn_dragon")]
        sw = sims[class_names.index("western_dragon")]
        westernization = float(sw - si)

    attr = attribute_scores(im, intended)

    rows.append({
        "image_path": path,
        "intended_label": intended,
        "intended_prompt": prompt_raw,
        "clip_pred": pred,
        "clip_prob_pred": float(np.max(probs)),
        "westernization_index": westernization,
        **attr
    })

res = pd.DataFrame(rows)
res.head()

Scoring:   0%|          | 0/3 [00:00<?, ?it/s]

,image_path,intended_label,intended_prompt,clip_pred,clip_prob_pred,westernization_index,serpentine_body,whiskers,horned_head,five_claws,auspicious_clouds,wings,bulky body,_adherence
0,image/dragon_001.jpg,cn_dragon,"A traditional Chinese dragon, no wings, deer-l...",cn_dragon,0.521811,-0.087300,0.506426,0.511520,0.498274,0.498982,0.501221,0.501923,0.499171,0.502502
1,image/dragon_002.jpg,cn_dragon,"A traditional Chinese dragon, no wings, deer-l...",cn_dragon,0.508979,-0.035919,0.508305,0.509557,0.499252,0.504134,0.499112,0.498924,0.497646,0.502419
2,image/dragon_003.jpg,cn_dragon,"A traditional Chinese dragon, no wings, deer-l...",western_dragon,0.529703,0.118951,0.493971,0.501448,0.491088,0.500115,0.484260,0.495066,0.498060,0.494858


# **汇总“总体指标”和“常被违背的属性”**

Deviation_Rate 就是“文化偏差率”（在 CLIP 视角下偏离目标类的比率）。
 Most Violated Attributes：均值最低的属性最容易被生成器忽略或做错（例如“五爪”“无翅”）。

In [15]:
# 偏差：CLIP 预测的类别 ≠ 你的 intended_label
res["is_deviation"] = (res["clip_pred"] != res["intended_label"]).astype(int)
deviation_rate = float(res["is_deviation"].mean()) if len(res) else 0.0

summary = {
    "N_images": int(len(res)),
    "Deviation_Rate(clip_pred!=intended)": deviation_rate,
}
if "_adherence" in res.columns:
    summary["Mean_Adherence"] = float(res["_adherence"].mean())
    summary["Median_Adherence"] = float(res["_adherence"].median())

# 统计各属性平均分（低=经常不满足/违背）
attr_cols = [c for c in res.columns if c not in ["image_path","intended_label","intended_prompt","clip_pred","clip_prob_pred","westernization_index","is_deviation"]]
violations = {c: float(res[c].mean()) for c in attr_cols if not c.startswith("_") and c in res.columns and res[c].notna().any()}
violations_sorted = sorted(violations.items(), key=lambda x: x[1])  # 从低到高

print("=== Overall Summary ===")
print(json.dumps(summary, indent=2))
print("\n=== Most Violated Attributes (lower mean = worse) ===")
for k, v in violations_sorted[:10]:
    print(f"{k:>20s}: {v:.3f}")

=== Overall Summary ===
{
  "N_images": 3,
  "Deviation_Rate(clip_pred!=intended)": 0.3333333333333333,
  "Mean_Adherence": 0.49992639110201886,
  "Median_Adherence": 0.5024186074733734
}

=== Most Violated Attributes (lower mean = worse) ===
   auspicious_clouds: 0.495
         horned_head: 0.496
          bulky body: 0.498
               wings: 0.499
          five_claws: 0.501
     serpentine_body: 0.503
            whiskers: 0.508


# **追加统计与导出：混淆比例矩阵（“文化偏差率”）**

核心文件：
intended_vs_pred_rowprop.csv = 你要的“文化偏差混淆比例矩阵”（每行归一化）。
intended_vs_pred_summary.json：总体偏差率 + 各类偏差率。

若csv里提供了 content_label（人工真实内容），还能得到：
content_vs_pred_rowprop.csv：CLIP 自身识别力；
intended_vs_content_rowprop.csv：生成器“真实跑题”比例（不依赖 CLIP）。

In [16]:
# 尝试并入人工内容标注（若你的 CSV 额外有 content_label 列）
try:
    raw = pd.read_csv(CSV_PATH)
    if "content_label" in raw.columns:
        res = res.merge(raw[["image_path","content_label"]], on="image_path", how="left")
    else:
        res["content_label"] = np.nan
except Exception:
    res["content_label"] = np.nan

outdir = "/content/clip_prompt_adherence_out"
os.makedirs(outdir, exist_ok=True)

def confusion_tables(df, row_col=("intended_label","clip_pred"), prefix="intended_vs_pred"):
    row, col = row_col
    sub = df[[row, col]].dropna()
    if sub.empty:
        return None

    counts = pd.crosstab(sub[row], sub[col], dropna=False)
    props  = pd.crosstab(sub[row], sub[col], normalize="index", dropna=False)
    counts.to_csv(f"{outdir}/{prefix}_counts.csv")
    props.to_csv(f"{outdir}/{prefix}_rowprop.csv")

    # 总体准确率与按类偏差率（= 1 - 行对角线）
    labels = counts.index.tolist()
    acc_per_class, bias_per_class = {}, {}
    for lab in labels:
        denom = counts.loc[lab].sum()
        diag  = counts.loc[lab, lab] if lab in counts.columns else 0
        acc   = float(diag / denom) if denom > 0 else 0.0
        acc_per_class[lab]  = acc
        bias_per_class[lab] = 1.0 - acc

    overall_acc = float(np.trace(counts.reindex(index=labels, columns=labels, fill_value=0).values) / counts.values.sum())
    overall_bias = 1.0 - overall_acc
    summary = {
        "overall_accuracy": overall_acc,
        "overall_bias_rate": overall_bias,          # = Deviation Rate
        "per_class_accuracy": acc_per_class,
        "per_class_bias_rate": bias_per_class,
    }
    with open(f"{outdir}/{prefix}_summary.json","w") as f:
        json.dump(summary, f, indent=2)
    return {"counts": counts, "props": props, "summary": summary}

# 1) 生成器“文化偏差率”： intended_label → clip_pred
tab_intended_pred = confusion_tables(
    res, row_col=("intended_label","clip_pred"), prefix="intended_vs_pred"
)

# 2)（可选）CLIP 自身识别力： content_label → clip_pred
tab_content_pred = None
if res["content_label"].notna().any():
    tab_content_pred = confusion_tables(
        res.dropna(subset=["content_label"]),
        row_col=("content_label","clip_pred"),
        prefix="content_vs_pred"
    )

# 3)（可选）生成器真实跑题： intended_label → content_label（不依赖 CLIP）
tab_intended_content = None
if res["content_label"].notna().any():
    tab_intended_content = confusion_tables(
        res.dropna(subset=["content_label"]),
        row_col=("intended_label","content_label"),
        prefix="intended_vs_content"
    )

# 导出逐图明细与摘要/属性排名
res.to_csv(f"{outdir}/per_image_scores.csv", index=False)
pd.DataFrame([summary]).to_csv(f"{outdir}/summary.csv", index=False)
pd.DataFrame(violations_sorted, columns=["attribute","mean_score"]).to_csv(f"{outdir}/violations_rank.csv", index=False)

print(f"Saved to: {outdir}")

Saved to: /content/clip_prompt_adherence_out


# **快速查看核心结果（Print）**

直接在 Colab 输出三张表（若无 content_label，后两张为 no data）

In [17]:
def pretty_summary(tab, title):
    if tab is None:
        print(f"\n=== {title} ===\n(no data)")
        return
    print(f"\n=== {title} ===")
    print("Counts:\n", tab["counts"])
    print("\nRow-normalized proportions (each row sums to 1):\n", tab["props"])
    print("\nSummary:\n", json.dumps(tab["summary"], indent=2))

pretty_summary(tab_intended_pred, "Intended (goal)  vs  CLIP Pred (cultural bias view)")
pretty_summary(tab_content_pred,  "Content (human) vs  CLIP Pred (CLIP accuracy view)")
pretty_summary(tab_intended_content, "Intended (goal)  vs  Content (generator drift)")


=== Intended (goal)  vs  CLIP Pred (cultural bias view) ===
Counts:
 clip_pred       cn_dragon  western_dragon
intended_label                           
cn_dragon               2               1

Row-normalized proportions (each row sums to 1):
 clip_pred       cn_dragon  western_dragon
intended_label                           
cn_dragon        0.666667        0.333333

Summary:
 {
  "overall_accuracy": 0.6666666666666666,
  "overall_bias_rate": 0.33333333333333337,
  "per_class_accuracy": {
    "cn_dragon": 0.6666666666666666
  },
  "per_class_bias_rate": {
    "cn_dragon": 0.33333333333333337
  }
}

=== Content (human) vs  CLIP Pred (CLIP accuracy view) ===
(no data)

=== Intended (goal)  vs  Content (generator drift) ===
(no data)
